### Урок 4



In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

__Установим MongoDb__


In [2]:
!pip install pymongo

In [3]:
import pymongo
from pymongo import MongoClient

__Спарсим данные__

In [4]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://vladivostok.hh.ru/search/vacancy?text=python&salary=&area=22&ored_clusters=true'

In [5]:
headers = {'User-Agent': USER_AGENT,}

In [10]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
        print('Парсинг завершен')
        return result
    else:
        print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
        result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
        link_next_page =  'https://vladivostok.hh.ru' + link_next_page['href']
    else:
        print('Парсинг завершен')
        return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
    sleep(0.1)
    vacancy_name = dom_vacancy.find('a').text

    vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
    if vacancy_salary:
        vacancy_salary = vacancy_salary.text
        min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
    else:
        min_salary, max_salary, currency_salary = None, None, None

    vacancy_link = dom_vacancy.find('a')['href']
  
    return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
    list_salary = vacancy_salary_text.replace('\u202f', '').split()
    for i in range(len(list_salary) - 1):
        if list_salary[i] == 'от':
            min_salary = int(list_salary[i + 1])
        elif list_salary[i] == 'до':
            max_salary = int(list_salary[i + 1])
        elif list_salary[i] == '–':
            min_salary = int(list_salary[i - 1])
            max_salary = int(list_salary[i + 1])
        currency_salary = list_salary[-1]

    return min_salary, max_salary, currency_salary

In [11]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://vladivostok.hh.ru/search/vacancy?text=python&salary=&area=22&ored_clusters=true
Cтраница №2, ссылка: https://vladivostok.hh.ru/search/vacancy?text=python&salary=&area=22&ored_clusters=true&page=1&hhtmFrom=vacancy_search_list
Парсинг завершен


__Запишем данные в базу__


In [12]:
client = MongoClient()

In [13]:
db = client.vacancies_python_hh

In [14]:
collection_vacancies_hh_ru = db.hh_ru

In [16]:
def cheak_and_save_vacancies_in_db(vacancies):
    for vacancy in vacancies:
        if not len(list(collection_vacancies_hh_ru.find({'vacancy_link': vacancy['vacancy_link']}))):
            collection_vacancies_hh_ru.insert_one(vacancy)

In [17]:
cheak_and_save_vacancies_in_db(result)

In [18]:
result_find = list(collection_vacancies_hh_ru.find())

In [19]:
len(result_find)

24

In [20]:
result_find

[{'_id': ObjectId('6404f7c55429eaf88a5d5502'),
  'vacancy_name': 'Аналитик данных (Junior)',
  'vacancy_salary': 'от 70\u202f000 руб.',
  'min_salary': 70000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://vladivostok.hh.ru/vacancy/76991488?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('6404f7c55429eaf88a5d5503'),
  'vacancy_name': 'Разработчик Python и SQL',
  'vacancy_salary': 'от 60\u202f000 руб.',
  'min_salary': 60000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://vladivostok.hh.ru/vacancy/76425036?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('6404f7c55429eaf88a5d5504'),
  'vacancy_name': 'Программист',
  'vacancy_salary': 'от 80\u202f000 руб.',
  'min_salary': 80000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://vladivostok.hh.ru/vacancy/74072093?from=vacancy_search_list&query=python',
  'vacancy_source